In [6]:
act_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
def_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2/'

num_traj = 20

import os

def select_eligible(dd, num_traj, pref=''):
    selected = {}
    traj = 1
    while True:
        traj_path = os.path.join(dd, str(traj), pref)
        if not os.path.isdir(traj_path):
            print(f'{traj_path} not found! Exiting')
            break
        
        imgs = os.listdir(os.path.join(traj_path, 'rgb'))
        print(f'{len(imgs)} in {traj_path}')
        if len(imgs) > 1000:
            selected[traj] = len(imgs)
        
        traj += 1
        if len(selected.keys()) >= num_traj:
            break
        
    return selected

In [7]:
act_eligible = select_eligible(act_dir, 20, 'default')

In [8]:
def_eligible = select_eligible(def_dir, 20)

In [9]:
for k,v in act_eligible.items():
    print(k, v)

1 1517
2 1677
3 1093
4 1797
5 1874
7 1895
8 1615
9 1593
10 1677
11 2038
12 2283
13 1820
14 1456
15 1615
16 1823
17 1825
18 1623
19 1327
20 2090
21 1816


In [10]:
for k,v in def_eligible.items():
    print(k, v)

1 1247
2 2502
3 2335
4 2353
5 2579
6 1915
7 1656
8 1278
9 2224
10 2539
11 2991
12 1110
13 1888
14 2357
15 1343
16 2744
17 1859
18 2724
19 2339
20 2568


In [ ]:
# now create new folder upto the min length
act_out = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019_sampled20'
def_out = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2_sampled20'

print(act_out, def_out)

def get_traj_length(ae, de, i):
    # get the ith k, v from each dict, return min length
    def get_ith_kv(d, i):
        x = 1
        for k, v in d.items():
            if x == i:
                return k,v
            x += 1
            
    
    ta_i, la_i = get_ith_kv(ae, i)
    td_i, ld_i = get_ith_kv(de, i)
    print(f'{ta_i, la_i, td_i, ld_i}')
    return min(la_i, ld_i), ta_i, td_i
    
import json
import glob

def acopyjson(a, b, num):
    with open(a, "r") as f:
        dat = json.load(f)
        
    dat2 = {}
    
    for x in range(num):
        dat2.update({x: dat[str(x)]})
        
    with open(b, "w") as fp:
        json.dump(dat2, fp)

from shutil import copyfile

def acopydir(a, b, ext, num):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b)   
        
    for x in range(num):
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,f))

def basic_sanity(traj_path):
    def ll(x, y, ext):
        return len(glob.glob(os.path.join(x, y, f'*{ext}')))
    # load json
    with open(os.path.join(traj_path, 'data.json'), 'r') as f:
        data = json.load(f)
    assert ll(traj_path, 'rgb', '.jpg') == ll(traj_path, 'seg', '.npy') == ll(traj_path, 'depth', '.npy') == len(data.keys())
    
def scopy(sd, st, dd, dt, l, pref=''):
    out_dir = os.path.join(dd, str(dt), pref)
    src_dir = os.path.join(sd, str(st), pref)
    
    print(f'sampling first {l} things from src_dir {src_dir}, out_dir {out_dir}')
    assert dd != sd
    
    if not os.path.isdir(out_dir):
        print(f'creating {out_dir}')
        os.makedirs(out_dir)
    
    acopydir(os.path.join(src_dir, 'rgb'), os.path.join(out_dir, 'rgb'), '.jpg', l)
    acopydir(os.path.join(src_dir, 'depth'), os.path.join(out_dir, 'depth'), '.npy', l)
    acopydir(os.path.join(src_dir, 'seg'), os.path.join(out_dir, 'seg'), '.npy', l)
    acopyjson(os.path.join(src_dir, 'data.json'), os.path.join(out_dir, 'data.json'), l)
    
    basic_sanity(out_dir)
    
for i in range(1, 21):
    minl, act_traj, def_traj = get_traj_length(act_eligible, def_eligible, i)
    print(f'{minl, act_traj, def_traj}')
    
    # now copy minl things 
    scopy(act_dir, act_traj, act_out, i, minl, 'default')
    scopy(def_dir, def_traj, def_out, i, minl)
    

/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019_sampled20 /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2_sampled20
(1, 1517, 1, 1247)
(1247, 1, 1)
sampling first 1247 things from src_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default, out_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019_sampled20/1/default
creating /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019_sampled20/1/default
sampling first 1247 things from src_dir /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2/1/, out_dir /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2_sampled20/1/
(2, 1677, 2, 2502)
(1677, 2, 2)
sampling first 1677 things from src_dir /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/2/default, out_dir /checkpoint/apratik/data_